# Basic Syntax


A decorator is basically a function (or a callable class) that takes an object and returns either that object or another object.

Examples of what it might do:

- add an attribute to the function
- wrap in higher-level function
- log

Decorators can be applied to:

- functions/methods
- classes


In [1]:
def decorator_function(original_function):

    def wrapper_function(*args, **kwargs):
        print(f'Before calling {original_function.__name__}')
        result = original_function(*args, **kwargs)
        print(f'After calling {original_function.__name__}')
        return result

    return wrapper_function


@decorator_function
def greet(name):
    print(f'Hello, {name}!')


greet('Alice')

Before calling greet
Hello, Alice!
After calling greet


# Built-In Decorators

@property: This decorator allows you to define a method as a property, providing a way to access and optionally modify an attribute value. It allows you to use a method like an attribute.

@staticmethod: This decorator defines a static method within a class. Static methods are bound to the class rather than an instance, and they can be called without creating an instance of the class.

@classmethod: This decorator defines a class method within a class. Class methods receive the class itself as the first argument (usually named cls) rather than the instance. They can be called on both the class and its instances.

@abstractmethod: This decorator is used to define abstract methods within an abstract base class (ABC). Abstract methods don't have an implementation in the base class and must be implemented by the derived classes.

@classmethod: This decorator is used in conjunction with the @abstractmethod decorator to define abstract class methods within an abstract base class. Abstract class methods are bound to the class and can be called on both the class and its instances.

@functools.wraps: This decorator is used when creating decorators to preserve the original function's metadata such as name, docstring, and parameter information. It ensures that the decorated function maintains its identity.


# Decorator Resolution

Note that decorators can come from anywhere and be resolved the normal way a name is.

```Python
@mymodule.submodule.myclass.mydecorator
def f():
    pass
```

A decorator can even come from something defined in the same class (see __Properties__ for example).

# Arguments

To make a decorator that takes arguments, you have to __add an extra factory layer__ because you need to make it so that calling `decorator_function(100, 200)` below returns a decorator.

Because of the way calling decorators works, you __cannot intermix__ calling with and without `()` - you have to know whether a decorator takes arguments or not and use it the right way.

If you really wanted to make a decorator that supports both (which is not the norm), you could branch by the type of the first arg inside.

In [6]:
def decorator_function(x, y):
    def decorator(fn):
        def wrapper_function(*args, **kwargs):
            print(f'Before calling {fn.__name__}')
            print(x, y)
            result = fn(*args, **kwargs)
            print(f'After calling {fn.__name__}')
            return result
        return wrapper_function

    return decorator

@decorator_function(100, 200)
def greet(name):
    print(f'Hello, {name}!')

greet('Alice')

Before calling greet
100 200
Hello, Alice!
After calling greet


In [7]:
def decorator_function(original_function):

    def wrapper_function(*args, **kwargs):
        print(f'Before calling {original_function.__name__}')
        result = original_function(*args, **kwargs)
        print(f'After calling {original_function.__name__}')
        return result

    return wrapper_function

@decorator_function()  # invalid!
def greet(name):
    print(f'Hello, {name}!')

greet('Alice')

TypeError: decorator_function() missing 1 required positional argument: 'original_function'

# Decorator Stacking

You can __vertically stack__ decorators to apply multiple ones to the same function. Because each one wraps the stuff below it, they will actually __apply in top-down order__ for things that happen before the function, and __bottom-up order__ for things that happen after.  In other words, they stack down and back up again.

In [13]:
def decorator_function(x, y):
    def decorator(original_function):
        def wrapper_function(*args, **kwargs):
            print(x, y)
            result = original_function(*args, **kwargs)
            print(x, y)
            return result
        return wrapper_function

    return decorator

@decorator_function(1, 2)
@decorator_function(3, 4)
def greet(name):
    print(f'Hello, {name}!')


greet('Alice')

1 2
3 4
Hello, Alice!
3 4
1 2


# Decorators Wrapping Decorators

Decorators can take other decorators to modify them before they get applied via `@`.  In this case, they are just functions taking function objects.  If they had arguments, you'd treat them as factories.

It is not the same thing as stacking.

In this example, we called decorator_function(decorator_function) which returns the wrapper ready to call the decorator again with before and after messages. Then we apply that to `greet`, which calls the wrapper, printing the 2 messages and calling the decorator to get the real decorator.

This example makes it look complicated, but a more concrete example from Django Rest Framework looks like this:
```Python
@method_decorator(login_required)
@method_decorator(cache_page(30))
```
where `@method_decorator` is being used to turn a decorator that is meant to apply to functions into a decorator that is meant to apply on class methods.

In [15]:
def decorator_function(original_function):

    def wrapper_function(*args, **kwargs):
        print(f'Before calling {original_function.__name__}')
        result = original_function(*args, **kwargs)
        print(f'After calling {original_function.__name__}')
        return result

    return wrapper_function

@decorator_function(decorator_function)
def greet(name):
    print(f'Hello, {name}!')

greet('Alice')

Before calling decorator_function
After calling decorator_function
Before calling greet
Hello, Alice!
After calling greet


# Inline Decorator Use

Because decorators are __just functions__ (or classes as the case may be), you can use them as such.

Note that there are __no @__ to be found here.

In [17]:
def decorator_function(original_function):

    def wrapper_function(*args, **kwargs):
        print(f'Before calling {original_function.__name__}')
        result = original_function(*args, **kwargs)
        print(f'After calling {original_function.__name__}')
        return result

    return wrapper_function

def greet(name):
    print(f'Hello, {name}!')

decorated_greet = decorator_function(greet)

greet('Alice')
print()
decorated_greet('Decorated Alice')

Hello, Alice!

Before calling greet
Hello, Decorated Alice!
After calling greet


In [18]:
def decorator_function(x, y):
    def decorator(fn):
        def wrapper_function(*args, **kwargs):
            print(f'Before calling {fn.__name__}')
            print(x, y)
            result = fn(*args, **kwargs)
            print(f'After calling {fn.__name__}')
            return result
        return wrapper_function

    return decorator

def greet(name):
    print(f'Hello, {name}!')
decorated_greet = decorator_function(100, 200)(greet)

greet('Alice')
print()
decorated_greet('Decorated Alice')

Hello, Alice!

Before calling greet
100 200
Hello, Decorated Alice!
After calling greet


# Generality of Decorator Syntax

1. `@` sign means the result of the __rest of this line__ is __called on the next thing__ and then __rebound to the name__
1. Anything that comes after the `@` is __just a python expression__.
1. Decorators with arguments are just __factory functions__ that return new decorators with closures inside.
1. Whatever you could do that way could also be done with variables by __directly calling the decorator__.
1. All of the above behavior can be derived from these simple facts.